In [2]:
pip install git+ssh://git@github.com/dorianbrown/rank_bm25.git

  Cloning ssh://****@github.com/dorianbrown/rank_bm25.git to c:\users\josipk~1\appdata\local\temp\pip-req-build-yuorub1v
  Created wheel for rank-bm25: filename=rank_bm25-0.2.1-py3-none-any.whl size=8574 sha256=2a9b29120ec2db2b3da393c173349178c45b559803209db16b0129854b2e1485
  Stored in directory: C:\Users\JOSIPK~1\AppData\Local\Temp\pip-ephem-wheel-cache-_rw6e9uh\wheels\b5\04\7a\a7a2d5c6113841f0d3bd39f9671cc7479f62bd33e2b63464d0
Successfully built rank-bm25
Note: you may need to restart the kernel to use updated packages.


In [34]:
from rank_bm25 import BM25Okapi
import glob
import sys
import os
import os.path 
import numpy as np
import string
from sklearn import preprocessing

In [20]:
## Importing bad and good bias words
################################### CHANGE ROUTE FOR CODE TO WORK ####################################################
os.chdir(r'C:\Users\JosipKoprcina\Desktop\IRProjectMovieScriptGenderBiasRetrieval')
bad_bias_words = open("BadBias.txt").read().split("\n")
good_bias_words = open("GoodBias.txt").read().split("\n")

## Initialzing needed dictionaries for single and paired words
bad_bias_unigrams = {}
bad_bias_bigrams = {}
good_bias_unigrams = {}
good_bias_bigrams = {}

## Adding bad bias words from basic text to dictionaries
for x in bad_bias_words:
    if(len(x.strip().split(" ")) == 2):
        bad_bias_bigrams[x.strip()] = 0
    else:
        bad_bias_unigrams[x.strip()] = 0
        
## Adding good bias words from basic text to dictionaries
for x in good_bias_words:
    if(len(x.strip().split(" ")) == 2):
        good_bias_bigrams[x.strip()] = 0
    else:
        good_bias_unigrams[x.strip()] = 0
        
## Importing movie scripts
################################### CHANGE ROUTE FOR CODE TO WORK ####################################################
os.chdir(r'C:\Users\JosipKoprcina\Desktop\IRProjectMovieScriptGenderBiasRetrieval\data')
myFiles = glob.glob('*.txt')
scripts = []
for file in myFiles:
    file = open(file,mode='r', encoding='utf-8-sig')
    scripts.append(file.read())
    file.close()

In [12]:
## Text Preprocessing

## Removing unneded white spaces and splitting scipts into tokens
scripts = [" ".join(doc.split()) for doc in scripts]
scripts = [doc.lower() for doc in scripts]
scripts = [doc.translate(str.maketrans("", "", string.punctuation)) for doc in scripts]
tokenized_corpus = [doc.split(" ") for doc in scripts]

## Creating pairs of tokens
tokenized_corpus_bigrams = []
for file in tokenized_corpus:
    current = []
    for i in range(len(file) - 1):
        current.append(file[i] + " " + file[i+1])
    tokenized_corpus_bigrams.append(current)

In [46]:
## Counting good and bad bias for each script and creating a weight list for all scripts
## Weight consists of number of different good/bad biases and the amount of times they occur (only second implemented for now)
bias_list = []

for i in range(len(scripts)):
    ## Increment dictionaries
    for word in tokenized_corpus[i]:
        if word in bad_bias_unigrams:
            if(bad_bias_unigrams[word] > 100):
                bad_bias_unigrams[word] += 0.01
            elif(bad_bias_unigrams[word] > 50):
                bad_bias_unigrams[word] += 0.1
            else:
                bad_bias_unigrams[word] += 1
        elif word in good_bias_unigrams:
            if(good_bias_unigrams[word] > 100):
                good_bias_unigrams[word] += 0.01
            elif(good_bias_unigrams[word] > 50):
                good_bias_unigrams[word] += 0.1
            else:
                good_bias_unigrams[word] += 1
    for word in tokenized_corpus_bigrams[i]:
        if word in bad_bias_bigrams:
            if(bad_bias_bigrams[word] > 100):
                bad_bias_bigrams[word] += 0.01
            elif(bad_bias_bigrams[word] > 50):
                bad_bias_bigrams[word] += 0.1
            else:
                bad_bias_bigrams[word] += 1
        elif word in good_bias_bigrams:
            if(good_bias_bigrams[word] > 100):
                good_bias_bigrams[word] += 0.01
            elif(good_bias_bigrams[word] > 50):
                good_bias_bigrams[word] += 0.1
            else:
                good_bias_bigrams[word] += 1
    
    ## Count bias and add to list of biases
    bad_bias = sum(bad_bias_unigrams.values()) + sum(bad_bias_bigrams.values())
    good_bias = sum(good_bias_unigrams.values()) + sum(good_bias_bigrams.values())
    bias =  good_bias-bad_bias*2
    bias_list.append(bias)
    
    ## Reset dictionaries to zero        
    bad_bias_unigrams = bad_bias_unigrams.fromkeys(bad_bias_unigrams, 0)
    bad_bias_bigrams = bad_bias_bigrams.fromkeys(bad_bias_bigrams, 0)
    good_bias_unigrams = good_bias_unigrams.fromkeys(good_bias_unigrams, 0)
    good_bias_bigrams = good_bias_bigrams.fromkeys(good_bias_bigrams, 0)

## Print bias_list for better visualization
bm25 = BM25Okapi(tokenized_corpus)

In [45]:
## Printing movie scripts ranked from most to least biased
bias_list_standardised = preprocessing.scale(bias_list)

bias = {}
for i in range(len(myFiles)):
    bias[myFiles[i]] = bias_list_standardised[i]
    

for w in sorted(bias, key=bias.get, reverse=False):
    print(w, bias[w])

Script_Master, The.txt -6.340371364490042
Script_Megamind.txt -4.7839159927252295
Script_Thunderbirds.txt -4.539339853993193
Script_Distinguished Gentleman, The.txt -4.198971394257626
Script_Producer, The.txt -4.137147981411448
Script_Shakespeare in Love.txt -4.056301979997212
Script_Sleepy Hollow.txt -3.8565648000326367
Script_Moonrise Kingdom.txt -3.7743600422920967
Script_12 Years a Slave.txt -3.7104984949564965
Script_Pride and Prejudice.txt -3.6948727972041646
Script_Barry Lyndon.txt -3.4788305413241067
Script_Titanic.txt -3.3429549086951407
Script_White Ribbon, The.txt -3.1900948219875556
Script_Panic Room.txt -3.130988921793956
Script_Ali.txt -3.1180807366942065
Script_Perfect Creature.txt -3.0915849883315536
Script_Jacket, The.txt -3.0881880975158316
Script_Star Wars_ Revenge of the Sith.txt -3.016853390385625
Script_Kung Fu Panda.txt -2.92921360733994
Script_Lord of the Rings_ Fellowship of the Ring, The.txt -2.921740447545346
Script_Rush Hour 2.txt -2.765483470022035
Script_C

Script_Platinum Blonde.txt 0.9778902089059774
Script_Fright Night (1985).txt 0.9846839905374258
Script_Rise of the Planet of the Apes.txt 0.9846839905374258
Script_Terminator 2_ Judgement Day.txt 0.9914777721688742
Script_Sixth Sense, The.txt 0.9982715538003224
Script_Catch Me If You Can.txt 1.001668444616048
Script_Return of the Apes.txt 1.0050653354317707
Script_Bourne Identity, The.txt 1.0186528986946672
Script_Alone in the Dark.txt 1.0254466803261157
Script_Patriot, The.txt 1.0254466803261157
Script_We Own the Night.txt 1.0254466803261157
Script_Color of Night.txt 1.0356373527732885
Script_Station West.txt 1.0390342435890123
Script_Gamer.txt 1.0458280252204604
Script_Tourist, The.txt 1.0526218068519089
Script_Fatal Instinct.txt 1.0594155884833572
Script_Jennifer Eight.txt 1.0594155884833572
Script_Antz.txt 1.0628124792990818
Script_Scream.txt 1.066209370114806
Script_Air Force One.txt 1.0730031517462537
Script_Saving Mr. Banks.txt 1.079796933377702
Script_Lincoln.txt 1.086590715009

In [38]:
## Importing queries
################################### CHANGE ROUTE FOR CODE TO WORK ####################################################
os.chdir(r'C:\Users\JosipKoprcina\Desktop\IRProjectMovieScriptGenderBiasRetrieval')
queries = open("Queries.txt").read().split("\n")

tokenized_queries = []
for query in queries:
    tokenized_queries.append(query.split(" "))

In [39]:
doc_scores = bm25.get_scores(tokenized_queries[4])
doc_scores_bias = [doc_scores[i] + bias_list[i] for i in range(len(doc_scores))]

movies = {}
movies_bias = {}
for i in range(len(myFiles)):
    movies[myFiles[i]] = doc_scores[i]
    movies_bias[myFiles[i]] = doc_scores_bias[i]

In [40]:
## Printing movie order of relevance to query before bias was added
for w in sorted(movies, key=movies.get, reverse=True):
    print(w, movies[w])

Script_Day the Earth Stood Still, The.txt 21.874880521596292
Script_Time Machine, The.txt 21.296580277001517
Script_Independence Day.txt 21.270326343562193
Script_Argo.txt 21.23931375625475
Script_Avengers, The (2012).txt 20.97362681114567
Script_Stuntman, The.txt 20.95943410595102
Script_Flash Gordon.txt 20.950139108880528
Script_Fifth Element, The.txt 20.94060234682184
Script_Mission to Mars.txt 20.925046320342123
Script_Arcade.txt 20.89845632110084
Script_After School Special.txt 20.890654712298023
Script_Doors, The.txt 20.88394027092391
Script_Soldier.txt 20.882944326726296
Script_Three Kings (Spoils of War).txt 20.801449268448728
Script_Broken Embraces.txt 20.783569364357156
Script_Serenity.txt 20.697397851436527
Script_Cinema Paradiso.txt 20.68877699485026
Script_Land of the Dead.txt 20.641048270866843
Script_Lost in Translation.txt 20.610017334749884
Script_Battle_ Los Angeles.txt 20.608358120578774
Script_Ed Wood.txt 20.58898582072075
Script_Star Trek_ First Contact.txt 20.5130

Script_Night Time (The Poltergeist Treatment).txt 16.21465129441646
Script_Under Fire.txt 16.211677953873604
Script_Blast from the Past, The.txt 16.210702029681237
Script_Mean Streets.txt 16.19521477350555
Script_500 Days of Summer.txt 16.19510829778982
Script_Pacifier, The.txt 16.189165285614145
Script_King of Comedy, The.txt 16.187594305619946
Script_Croupier.txt 16.1841418655481
Script_Revolutionary Road.txt 16.18119826137512
Script_Burning Annie.txt 16.177736792505026
Script_Easy A.txt 16.17728844477432
Script_Casino.txt 16.168982520163468
Script_Yes Man.txt 16.16661780209206
Script_Case 39.txt 16.166199614960018
Script_I am Sam.txt 16.164492978998236
Script_Bachelor Party.txt 16.164393974968213
Script_Jennifer's Body.txt 16.1628393474285
Script_Lincoln.txt 16.161870476084253
Script_Pride and Prejudice.txt 16.160518487344316
Script_Best Exotic Marigold Hotel, The.txt 16.139713456219365
Script_Frozen River.txt 16.1371137509079
Script_Absolute Power.txt 16.134242048904802
Script_Yout

Script_Three Men and a Baby.txt 14.17584272966917
Script_Apartment, The.txt 14.157083263227989
Script_Single White Female.txt 14.153076374358422
Script_Judge Dredd.txt 14.137572245471285
Script_Friday the 13th.txt 14.133232742574922
Script_Fright Night.txt 14.127468678984142
Script_Ugly Truth, The.txt 14.120375080517375
Script_Nashville.txt 14.089928805506887
Script_House of 1000 Corpses.txt 14.073764720346775
Script_Jane Eyre.txt 14.07238309219597
Script_Shallow Grave.txt 14.070185134983621
Script_Tristan and Isolde.txt 14.064764879332198
Script_Kramer vs Kramer.txt 14.059417377607343
Script_Arsenic and Old Lace.txt 14.048709043902404
Script_Cast Away.txt 14.044154246134541
Script_Five Easy Pieces.txt 14.038982550159444
Script_Visitor, The.txt 14.00751186214725
Script_Clueless.txt 14.003783440506012
Script_Neverending Story, The.txt 13.995648124695713
Script_Cruel Intentions.txt 13.989678262798517
Script_Army of Darkness.txt 13.989328663838675
Script_Face Off.txt 13.98644665830731
Scr

In [41]:
## Printing movie order of relevance to query after bias was added
for w in sorted(movies_bias, key=movies_bias.get, reverse=True):
    print(w, movies_bias[w])

Script_Arcade.txt 20.89845632110084
Script_Soldier.txt 20.882944326726296
Script_Three Kings (Spoils of War).txt 20.801449268448728
Script_Star Trek_ First Contact.txt 20.513084563915022
Script_Alien_ Resurrection.txt 20.4202093773808
Script_Max Payne.txt 20.124689931828435
Script_Gravity.txt 19.486906826662324
Script_Alien vs. Predator.txt 19.12567171299758
Script_Mission to Mars.txt 19.125046320342122
Script_Beach, The.txt 19.04156482590812
Script_Predator.txt 18.92279846708745
Script_Serenity.txt 18.897397851436526
Script_Ronin.txt 18.84654651833736
Script_Dances with Wolves.txt 18.726672953272995
Script_Rise of the Planet of the Apes.txt 18.434348452901606
Script_Avatar.txt 18.39528512296526
Script_Source Code.txt 18.046764511365748
Script_Whiteout.txt 18.0445515475555
Script_Save the Last Dance.txt 17.988227833175507
Script_Cider House Rules, The.txt 17.941635490275814
Script_Solaris.txt 17.92363924762256
Script_Mud.txt 17.8093057143045
Script_Logan's Run.txt 17.782144063532744
Sc

Script_Social Network, The.txt -18.727442025763278
Script_Ghost World.txt -19.082042844031147
Script_Hot Tub Time Machine.txt -19.12102707264433
Script_Peggy Sue Got Married.txt -19.136492835124237
Script_Sideways.txt -19.43704071881681
Script_Blood and Wine.txt -19.466924501141765
Script_Change-Up, The.txt -19.539355619352218
Script_True Lies.txt -19.630877153447955
Script_Birthday Girl.txt -19.659876833431596
Script_Priest.txt -19.74548359334529
Script_Batman.txt -19.94268414811758
Script_Manchurian Candidate, The.txt -20.06966620582652
Script_Ugly Truth, The.txt -20.079624919482626
Script_Last Chance Harvey.txt -20.22869200223997
Script_Spanglish.txt -20.269664012259486
Script_Imaginarium of Doctor Parnassus, The.txt -20.330153138953094
Script_Doors, The.txt -20.516059729076087
Script_White Christmas.txt -20.524620066360143
Script_Two For The Money.txt -20.56690859977965
Script_Wall-E.txt -20.567089735334214
Script_Lord of the Rings_ Return of the King.txt -20.98730352659396
Script_